### 본격적으로 연습한 코드를 세련되게 다듬어 보는 중

Input > Output이 너무 달라서 욕함

보통의 경우 / 하나의 변수(시계열), n 일간의 데이터 학습 > 1개의 정답

이번 경우 / 일단 변수가 5~6개, 거기에 n일 + 정답이 1개가 아님, 2일치로 쭉 뽑아내야 함

In [11]:
import pandas as pd
from google.colab import drive
from pandas import Series, DataFrame
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import seaborn as sns
from pandas import DataFrame
from pandas import concat
import os
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [12]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')

In [14]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset = dataset[what_to_use]
  return dataset

In [15]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 함수
def series_to_supervised(data, n_in=1, n_out=1, target = 'TARGET', dropnan=True):
    df = DataFrame(data)
    df.drop(target, axis = 1, inplace=True)
    df2 = DataFrame(data[target])
    cols, names = list(), list()
    n_vars = 1 if type(df) is list else df.shape[1]
    n_vars2 = 1 if type(df2) is list else df2.shape[1]
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df2.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [16]:
def seperator(dataset):
  
  # series_to_supervised 함수를 지나며 전처리를 하면 하나의 row에 이전 7일간의 데이터가 일렬로 들어가고(train), 여기에 target으로 미래 96일 발전량이 따라 붙는다.
  # 이 둘이 한 줄에 있으므로 적절한 지점에서 잘라서 train_X와 train_y로 사용한다.
  X = dataset.iloc[:, :n_obs]
  y = dataset.iloc[:, -future_window:]
  
  # validation을 해야하니까, train으로 준 데이터를 train/test로 자른다.
  # 사용가능한 연도가 3개년이고, 대충 train에 2년치, test에 1년치를 주었다 (7:3)
  # 42 = ultimate answer to life the universe and everything
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

  # 학습과 검증 데이터를 원하는 모양([1,4,2]) 이런 식으로 바꾸는 함수
  #### 처음 모양 (36490, 2016)
  # 2년치가 데이터(730일 * 하루에 48틱 = 36490개)로 있고, 이전 일주일 데이터 7*48 = 336인데 사용한 변수가 6개라면 336*6이다.
  # 일단 2016개의 데이터를 틱별로(30분 단위로) 잘라준다
  # 한 틱에 6개의 변수가 들어간다면 이를 n_features로 반영
  
  #### 수정할 모양 (36490, 336, 6)
  # 6개 특징이 30분 단위로 336개(일주일) 준비되어 있다.
  # 이런 데이터가 36490개 있어서 학습에 사용할 수 있다.

  # y 데이터는 1개 변수(발전량)만 나오고, 미래 2일 (2*48 = 96틱)이므로
  # 형태 ('총 데이터의 개수', '미래/과거의 길이', '사용한 변수의 개수')
  # == (36490, 96, 1) 으로 reshape 한다.

  train_X = X_train.values
  train_X = train_X.reshape(train_X.shape[0],-1,n_features)
  train_y = y_train.values
  train_y = train_y.reshape(train_y.shape[0],future_window)

  test_X = X_test.values
  test_X = test_X.reshape(test_X.shape[0],-1,n_features)
  test_y = y_test.values
  test_y = test_y.reshape(test_y.shape[0],future_window)

  return train_X, train_y, test_X, test_y

In [17]:
def data_maker(train_X, train_y, test_X, test_y):
  train_data = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
  val_data = TensorDataset(torch.from_numpy(test_X), torch.from_numpy(test_y))
  train_set = DataLoader(train_data, batch_size = batches, shuffle=True)
  test_set = DataLoader(val_data, batch_size = batches, shuffle=True)
  dateiter = iter(train_set)
  tx,ty = dateiter.next()
  print("Final Shape is :")
  print(tx.size())
  print(ty.size())
  return train_set, test_set

In [18]:
############ 파라미터 #####################
# 하루의 틱
ticks = 48
# 예측에 사용할 일수
days = 7
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']
# 사용할 변수
what_to_use = ['Time', 'DHI','DNI','RH','T','TARGET']

n_features = len(what_to_use) - 1
n_obs = n_days * n_features

# 한 번에 뭉테기로 투입할 자료의 양
batches = 100
# 몇 번이나 반복하여 학습할 것인다.
epoch = 200

In [19]:
def preprocessing(dataset):
  dataset = drop_clms(dataset)
  dataset = series_to_supervised(dataset, n_days, future_window, target='TARGET')
  train_X, train_y, test_X, test_y = seperator(dataset)
  train_set, test_set = data_maker(train_X, train_y, test_X, test_y)
  return train_set, test_set

In [20]:
train_set, test_set = preprocessing(raw_data)

Final Shape is :
torch.Size([100, 336, 5])
torch.Size([100, 96])


In [10]:
# 일단 변수는 5개, timestep은 336개(7일) 이므로, input_size는 5가 된다.
input_size = n_features

# hidden_size는 input_size를 받아서 출력하는 크기를 나타낸다.
# 미래 2일, 96개를 출력해야 하므로 hidden_size = 96
hidden_size = future_window

# Layer는 임의로 4개쯤 깔아준다.
layers = 4

In [ ]:
## LSTM 함수 생성


class LSTM(nn.Module):
    
    # __init__은 class 즉, LSTM의 변수값을 정의하며 시작하는 역할을 한다.
    # self.input_size = 10이라고 하면, 이 함수의 input_size (=self의 input_size)를 10으로 정의하겠다는 의미
    
    def __init__(self,input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        
        self.rnn = nn.LSTM(
            input_size = input_size, 
            hidden_size = hidden_size, 
            num_layers = layers, 
            batch_first = True,
            bidirectional = True
        )
# Linear 뒤에가 출력값.
# bidirectional을 쓰기 때문에(역전파), 결과값 96개, 역전파 96개 즉 future_window의 두 배
        self.layers = nn.Sequential(
            nn.ReLU(),
            nn.Linear(hidden_size*2, hidden_size),
        )
        
    def forward(self, x):
        y,_ = self.rnn(x)
        y = self.layers(y)
        return y
    
model = LSTM(input_size, hidden_size)

In [ ]:
model

LSTM(
  (rnn): LSTM(5, 96, num_layers=4, batch_first=True, bidirectional=True)
  (layers): Sequential(
    (0): ReLU()
    (1): Linear(in_features=192, out_features=96, bias=True)
  )
)

In [ ]:
# 손실함수
# Quantile 별로 구해야하니까, q 값에 quantile을 넣어야한다.

def tilted_loss(q,y,f):
    e = (y-f)
    return torch.mean(torch.max(q*e, (q-1)*e), axis=-1)

# loss & optimizer setting
# 원래 critertion이 설정된 손실함수 이지만, quantile 값을 구해야 하므로 대신 위에 새로 정의한 손실함수 사용
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# start training
for i in range(epoch):
    for i, (train, train_target) in enumerate(train_set):
      train = train.float()
      train_target = train_target.float()

      model.train()
      outputs = model(train)
      loss = tilted_loss(0.5, outputs[2][1], train_target[2])
      
      optimizer.zero_grad()

      loss.backward()
      optimizer.step()
      if i%500 == 0:
          result = outputs.data.numpy().argmax(axis=2)
          result_str = ''.join([char_set[c] for c in np.squeeze(result)])
          print(i, "loss: ", loss.item(), "\nprediction: ", result, "\ntrue Y: ", y_data, "\nprediction str: ", result_str,"\n")

RuntimeError: ignored

In [ ]:
for i, (train, train_target) in enumerate(train_set):
  print(train_target)

In [ ]:
for i, (train, train_target) in enumerate(train_set):
  train = train.float()
  train_target = train_target.float()

  model.train()
  outputs = model(train)
  #print(train_target[2])
  print(outputs[-1].shape)

torch.Size([336, 96])
torch.Size([336, 96])


KeyboardInterrupt: ignored